![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [2]:
-- Run this code to view the data in students
SELECT * 
FROM students;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24.0,4.0,5.0,Long,3.0,Average,5.0,High,,Yes,No,No,No,0.0,Min,34.0,23.0,9.0,11.0,8.0,11.0,2.0,27.0,91.0,5.0,5.0,6.0,3.0,2.0,1.0,4.0,1.0,3.0,4.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28.0,5.0,1.0,Short,4.0,High,4.0,High,,No,No,No,No,2.0,Min,48.0,8.0,7.0,5.0,4.0,3.0,2.0,10.0,39.0,7.0,7.0,7.0,4.0,4.0,4.0,4.0,1.0,1.0,1.0,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25.0,4.0,6.0,Long,4.0,High,4.0,High,Yes,Yes,No,No,No,2.0,Min,41.0,13.0,4.0,7.0,6.0,4.0,3.0,14.0,51.0,3.0,3.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29.0,5.0,1.0,Short,2.0,Low,3.0,Average,No,No,No,No,No,3.0,Min,37.0,16.0,10.0,10.0,8.0,6.0,4.0,21.0,75.0,5.0,5.0,5.0,5.0,5.0,2.0,2.0,2.0,4.0,4.0,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28.0,5.0,1.0,Short,1.0,Low,3.0,Average,Yes,No,No,No,No,3.0,Min,37.0,15.0,12.0,5.0,8.0,7.0,4.0,31.0,82.0,5.0,5.0,5.0,2.0,5.0,2.0,5.0,5.0,4.0,4.0,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,140,,,,,,,,,
282,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,131,,,,,,,,,
283,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66,202,,,,,,,,,
284,,,,,NaN,NaN,NaN,,NaN,,NaN,,,,,,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,207,,,,,,,,,


In order to successfully complete this task, we will focus only on the so-called 'useful' variables, even though other variables such as gender, family, and social situation might be of interest. Thus, we will only focus on the following variables: inter_dom, stay, todep, tosc, and toas. 

**DATA TYPE**

In [3]:
-- Check the data type for selected fields
SELECT
	column_name,
	data_type
FROM INFORMATION_SCHEMA.COLUMNS
WHERE 
	column_name IN ('inter_dom', 'stay', 'todep', 'tosc', 'toas')
	AND table_name = 'students';

,column_name,data_type
0,stay,integer
1,todep,integer
2,tosc,integer
3,toas,integer
4,inter_dom,text


*No problem*

**NATIONALITY IMPACT ON STUDENTS' MENTAL HEALTH**

**1. Field content**

In [4]:
--  Create the temporary inter_dom_analysis table
DROP TABLE IF EXISTS inter_dom_analysis;

CREATE TEMP TABLE inter_dom_analysis AS
SELECT 
	inter_dom,
	COUNT(*) AS count,
	ROUND(COUNT(*)*100.0/SUM(COUNT(*)) OVER (), 2) AS percentage
FROM students 
GROUP BY inter_dom; 

SELECT *
FROM inter_dom_analysis;

,inter_dom,count,percentage
0,Dom,67,23.43
1,,18,6.29
2,Inter,201,70.28


*TO NOTE: We have 18 missing values.*

In [5]:
-- inter_dom missing values
SELECT *
FROM students 
WHERE inter_dom NOT IN ('Dom', 'Inter');

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.0,222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,249.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,42.0,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,128.0,137.0,66.0,61.0,30.0,46.0,19.0,65.0,21.0,45.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,54.0,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,140.0,131.0,202.0,207.0,238.0,222.0,249.0,203.0,247.0,223.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,128.0,137.0,66.0,61.0,30.0,46.0,19.0,65.0,21.0,45.0


*The 18 rows lacking a value for this field are ultimately of no use for the project. Therefore, they will not be taken into account moving forward.*

**2. Correlation between 'inter_dom' and depression (PHQ-9 test)**

In [6]:
-- Missing values for the 3 fields of students' mental health
SELECT
	inter_dom,
	todep,
	toas,
	tosc
FROM students 
WHERE 
	inter_dom IN ('Dom', 'Inter') AND
	(todep IS NULL OR toas IS NULL OR tosc IS NULL);

,inter_dom,todep,toas,tosc


General depression level

In [7]:
-- General depression level
SELECT
	todep,
	COUNT(*) AS todep_level_count,
	ROUND((COUNT(*)*100.0/SUM(COUNT(*)) OVER()), 2) AS todep_level_percentage
FROM students 
WHERE inter_dom IN ('Dom', 'Inter')
GROUP BY todep
ORDER BY todep;

,todep,todep_level_count,todep_level_percentage
0,0,13,4.85
1,1,8,2.99
2,2,15,5.60
3,3,17,6.34
4,4,12,4.48
5,5,13,4.85
6,6,23,8.58
7,7,22,8.21
8,8,20,7.46
9,9,29,10.82


Nationality influence on students' depression level

In [8]:
-- Define a table of all possible depression levels from 0 to 25
WITH all_levels AS (
    SELECT generate_series(0, 25) AS level
),

-- CTEs for depression level distribution in 'Dom' and 'Inter' students
dom_phq AS (
	SELECT
		todep,
		ROUND((COUNT(*)*100/SUM(COUNT(*)) OVER()), 2) AS dom_phq_percentage
	FROM students 
	WHERE inter_dom = 'Dom'
	GROUP BY todep
	ORDER BY todep
),

inter_phq AS (
	SELECT
		todep,
		ROUND((COUNT(*)*100/SUM(COUNT(*)) OVER()), 2) AS inter_phq_percentage
	FROM students 
	WHERE inter_dom = 'Inter'
	GROUP BY todep
	ORDER BY todep
)

-- Main request
SELECT
	al.level AS level,
	COALESCE(dp.dom_phq_percentage, 0) AS dom_phq_percentage,		-- Fill by 0 if todep level is null
	COALESCE(ip.inter_phq_percentage, 0) AS inter_phq_percentage	-- Fill by 0 if todep level is null
FROM all_levels as al
LEFT JOIN dom_phq AS dp
	ON al.level = dp.todep
LEFT JOIN inter_phq AS ip
	ON al.level = ip.todep
ORDER BY level;

,level,dom_phq_percentage,inter_phq_percentage
0,0,5.97,4.48
1,1,2.99,2.99
2,2,7.46,4.98
3,3,2.99,7.46
4,4,1.49,5.47
5,5,2.99,5.47
6,6,5.97,9.45
7,7,10.45,7.46
8,8,5.97,7.96
9,9,13.43,9.95


*The depression level of students does not appear to be influenced by whether the student is of Japanese nationality or not.*

**3. Correlation between students' nationality and social connectedness (SCS test)**

In [9]:
-- General social connectedness level
SELECT
	tosc,
	COUNT(*) AS scs_count,
	ROUND((COUNT(*)*100/SUM(COUNT(*)) OVER()), 2) AS scs_percentage
FROM students 
WHERE inter_dom IN ('Dom', 'Inter')
GROUP BY tosc
ORDER BY tosc;	

,tosc,scs_count,scs_percentage
0,8,1,0.37
1,11,2,0.75
2,13,1,0.37
3,14,1,0.37
4,15,1,0.37
5,16,1,0.37
6,17,1,0.37
7,18,2,0.75
8,19,2,0.75
9,20,1,0.37


In [10]:
-- Statistical analysis
SELECT 
	inter_dom,
	MIN(tosc) AS min_scs,
	MAX(tosc) AS max_scs,
	ROUND(AVG(tosc), 2) AS mean_scs,
	ROUND(percentile_disc(0.5) WITHIN GROUP (ORDER BY tosc), 2) AS median_scs
FROM students 
WHERE inter_dom IN ('Dom', 'Inter')
GROUP BY inter_dom;

,inter_dom,min_scs,max_scs,mean_scs,median_scs
0,Dom,8,48,37.64,39.0
1,Inter,11,48,37.42,40.0


**4. Correlation between students' nationality and acculturative stress (ASISS test)**

In [11]:
-- General acculturative stress
SELECT
	toas,
	COUNT(*) AS asiss_count,
	ROUND((COUNT(*)*100/SUM(COUNT(*)) OVER()), 2) AS asiss_percentage
FROM students 
WHERE inter_dom in ('Dom', 'Inter')
GROUP BY toas
ORDER BY toas;

,toas,asiss_count,asiss_percentage
0,36,13,4.85
1,37,4,1.49
2,39,3,1.12
3,40,3,1.12
4,41,4,1.49
...,...,...,...
75,127,1,0.37
76,129,1,0.37
77,132,1,0.37
78,133,1,0.37


In [12]:
-- Statistical analysis
SELECT
	inter_dom,
	MIN(toas) AS min_asiss,
	MAX(toas) AS max_asiss,
	ROUND(AVG(toas), 2) AS mean_asiss,
	ROUND(stddev(toas), 2) AS sd_asiss,
	ROUND(percentile_disc(0.5) WITHIN GROUP (ORDER BY toas), 2) AS median_asiss
FROM students 
WHERE inter_dom IN ('Dom', 'Inter')
GROUP BY inter_dom;

,inter_dom,min_asiss,max_asiss,mean_asiss,sd_asiss,median_asiss
0,Dom,36,112,62.84,20.24,64.0
1,Inter,36,145,75.56,22.56,75.0


*Not surprisingly, international students exhibit higher mean and median scores on the ASISS questionnaire. More in-depth statistical analyses could be conducted using Python...*

**5. Conclusion for correlations between students' nationality and the 3 fields of students' mental health.**

Foreign students exhibit higher ASISS scores compared to Japanese students, while scores on the PHQ-9 and SCS questionnaires are similar.

The next point to be addressed will be the impact of the duration of stay on the mental health of foreign students. Although not requested, we will also delve into the relationship between their language proficiency and mental health.

**STAY LENGTH IMPACT ON STUDENTS' MENTAL HEALTH**

**1. Stay content for international student**

In [13]:
-- Stay content for only international student
SELECT
	stay,
	COUNT(*) AS count_stay,
	ROUND((COUNT(*)*100/SUM(COUNT(*)) OVER()), 2) AS percentage_stay
FROM students 
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay;

,stay,count_stay,percentage_stay
0,1,95,47.26
1,2,39,19.40
2,3,46,22.89
3,4,14,6.97
4,5,1,0.50
5,6,3,1.49
6,7,1,0.50
7,8,1,0.50
8,10,1,0.50


*Over 90% of the surveyed foreign students have been in Japan for 1 to 3 years. While we will conduct a general statistical analysis, it might be interesting for a more in-depth analysis to group stays of more than 4 years into a single category...*

**2. Correlation between the stay length and depression (PHQ-9 test)**

With no data manipulation

In [14]:
-- Statistical analysis with no data manipulation
SELECT
	stay,
	MIN(todep) AS min_phq,
	MAX(todep) AS max_phq,
	ROUND(AVG(todep), 2) AS mean_phq,
	ROUND(STDDEV(todep), 2) AS std_phq,
	ROUND(percentile_disc(0.5) WITHIN GROUP (ORDER BY todep), 2) AS median_phq
FROM students 
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay;

,stay,min_phq,max_phq,mean_phq,std_phq,median_phq
0,1,0,25,7.48,4.61,7.0
1,2,0,21,8.28,4.90,8.0
2,3,0,24,9.09,5.57,9.0
3,4,0,14,8.57,4.48,7.0
4,5,0,0,0.00,NaN,0.0
5,6,2,10,6.00,4.00,6.0
6,7,4,4,4.00,NaN,4.0
7,8,10,10,10.00,NaN,10.0
8,10,13,13,13.00,NaN,13.0


In [15]:
-- Correlation matrix with no data manipulation
SELECT 
    CORR(stay, todep) AS stay_depression_correlation
FROM students 
WHERE inter_dom = 'Inter';

,stay_depression_correlation
0,0.072969


With data manipulation

In [16]:
-- Statistical analysis with data manipulation (4 different stay lengths)
WITH grouped_stay AS (
	SELECT
		todep,
		CASE WHEN stay >= 4 THEN '4 and above'
		ELSE CAST(stay AS VARCHAR) END AS stay_length
	FROM students 
	WHERE inter_dom = 'Inter'
)

SELECT
	stay_length,
	MIN(todep) AS min_phq,
	MAX(todep) AS max_phq,
	ROUND(AVG(todep), 2) AS mean_phq,
	ROUND(STDDEV(todep), 2) AS std_phq,
	ROUND(percentile_disc(0.5) WITHIN GROUP (ORDER BY todep), 2) AS median_phq
FROM grouped_stay
GROUP BY stay_length
ORDER BY stay_length;

,stay_length,min_phq,max_phq,mean_phq,std_phq,median_phq
0,1,0,25,7.48,4.61,7.0
1,2,0,21,8.28,4.90,8.0
2,3,0,24,9.09,5.57,9.0
3,4 and above,0,14,7.86,4.57,7.0


In [17]:
-- Correlation matrix with data manipulation
WITH grouped_stay AS (
	SELECT
		todep,
		CASE WHEN stay >= 4 THEN '4'			-- Only 4 to convert into integer for correlation matrix
		ELSE CAST(stay AS INTEGER) 
		END AS stay_length
	FROM students 
	WHERE inter_dom = 'Inter'
)

SELECT
	CORR(stay_length::INTEGER, todep) AS stay_depression_correlation
FROM grouped_stay;

,stay_depression_correlation
0,0.089741


CONCLUSION: 

The results are very similar to those previously obtained. Therefore, there is no correlation between the length of stay and the level of depression.

**3. Correlation between the stay length and the social connectedness (SCS test)**

*We previously saw that long durations of stay concerned very few students (only one student in some cases). Thus, as with the PHQ-9 test score, stays of 4 years or more will be grouped together.*

In [18]:
-- Statistical analysis with data manipulation
WITH grouped_stay AS (
	SELECT
		tosc,
		CASE WHEN stay >= 4 THEN '4'
		ELSE CAST(stay AS VARCHAR)
		END AS stay_length
	FROM students 
	WHERE inter_dom = 'Inter'
)

SELECT
	stay_length,
	MIN(tosc) AS min_scs,
	MAX(tosc) AS max_scs,
	ROUND(AVG(tosc), 2) AS mean_scs,
	ROUND(STDDEV(tosc), 2) AS std_scs,
	ROUND(percentile_disc(0.5) WITHIN GROUP (ORDER BY tosc), 2) AS median_scs
FROM grouped_stay
GROUP BY stay_length
ORDER BY stay_length;

,stay_length,min_scs,max_scs,mean_scs,std_scs,median_scs
0,1,11,48,38.11,8.53,40.0
1,2,11,48,37.08,10.75,41.0
2,3,13,48,37.13,9.19,40.0
3,4,17,48,35.57,8.73,36.0


In [19]:
-- Correlation matrix with data manipulation
WITH grouped_stay AS (
	SELECT
		tosc,
		CASE WHEN stay >= 4 THEN '4'
		ELSE CAST(stay AS INTEGER)
		END AS stay_length
	FROM students 
	WHERE inter_dom = 'Inter'
)

SELECT
	CORR(stay_length::INTEGER, tosc) AS stay_scs_correlation
FROM grouped_stay;

,stay_scs_correlation
0,-0.080616


CONCLUSION: 

Although this analysis reveals no correlation between the length of stay and the lack of social connections, there seems to be an improvement in the SCS test score starting from the fourth year of stay. To confirm this finding, it would be interesting to increase the study's sample size.

**4. Correlation between the stay length and the acculturative stress (ASISS test)**

In [20]:
-- Statistical analysis with data manipulation
WITH grouped_stay AS (
	SELECT
		toas,
		CASE WHEN stay >= 4 THEN '4'
		ELSE CAST(stay AS VARCHAR)
		END AS stay_length
	FROM students 
	WHERE inter_dom = 'Inter'
)

SELECT
	stay_length,
	MIN(toas) AS min_asiss,
	MAX(toas) AS max_asiss,
	ROUND(AVG(toas), 2) AS mean_asiss,
	ROUND(STDDEV(toas), 2) AS std_asiss,
	ROUND(percentile_disc(0.5) WITHIN GROUP (ORDER BY toas), 2) AS median_asiss
FROM grouped_stay
GROUP BY stay_length
ORDER BY stay_length;

,stay_length,min_asiss,max_asiss,mean_asiss,std_asiss,median_asiss
0,1,36,145,72.80,20.46,70.0
1,2,36,127,77.67,23.43,81.0
2,3,36,133,78.00,22.96,78.0
3,4,36,129,78.81,28.66,83.0


In [21]:
-- Correlation matrix with data manipulation
WITH grouped_stay AS (
	SELECT
		toas,
		CASE WHEN stay >= 4 THEN '4'
		ELSE CAST(stay AS INTEGER)
		END AS stay_length
	FROM students 
	WHERE inter_dom = 'Inter'
)

SELECT
	CORR(stay_length::INTEGER, toas) AS stay_scs_correlation
FROM grouped_stay;

,stay_scs_correlation
0,0.106842


CONCLUSION: 

Similarly to the other two tests, there is no correlation between the length of stay and acculturative stress. However, it appears that after 2 years, the ASISS test score worsens!

**5. Conclusion for correlations between students' stay length and the 3 fields of students' mental health.**

While there is no correlation between the level of depression (PHQ-9 test) and the length of stay, there seem to be cut-offs for social connections and acculturative stress. Starting from 4 years, social connections seem to improve, which is positive. Conversely, from 2 years onwards, acculturative stress seems to deteriorate...

The study showed that social connectedness and acculturative stress were predictive of depression. It is up to us to verify this...

**SOCIAL CONNECTEDNESS AND ACCULTURATIVE STRESS, MARKERS OF DEPRESSION?**

In [22]:
-- Correlation matrix
DROP TABLE IF EXISTS correlations;

CREATE TEMP TABLE correlations AS 
	SELECT
		'depression':: VARCHAR AS measure,
		CORR(todep, todep) AS depression,
		CORR(todep, tosc) AS social_connectedness,
		CORR(todep, toas) AS acculturative_stress
	FROM students 
	WHERE inter_dom = 'Inter';
	
INSERT INTO correlations
SELECT
	'social_connectedness'::VARCHAR AS measure,
	CORR(todep, tosc) AS depression,
	CORR(tosc, tosc) AS social_connectedness,
	CORR(tosc, toas) AS acculturative_stress
FROM students 
WHERE inter_dom = 'Inter';

INSERT INTO correlations
SELECT
	'acculturative_stress'::VARCHAR AS measure,
	CORR(toas, todep) AS depression, 
	CORR(toas, tosc) AS social_connectedness,
	CORR(toas,toas) AS acculturative_stress
FROM students 
WHERE inter_dom = 'Inter';

SELECT 
	measure,
	ROUND(depression::NUMERIC, 2) AS depression,
	ROUND(social_connectedness::NUMERIC, 2) AS social_connectedness, 
	ROUND(acculturative_stress::NUMERIC, 2) AS acculturative_stress
FROM correlations;

,measure,depression,social_connectedness,acculturative_stress
0,depression,1.00,-0.54,0.41
1,social_connectedness,-0.54,1.00,-0.58
2,acculturative_stress,0.41,-0.58,1.00


CONCLUSION: 

According to this correlation table, it indeed seems that social connectedness and acculturative stress could be predictive of depression.

We have thus answered all the questions posed and have confirmed that social connectedness and acculturative stress could be predictive markers of depression.

Other analyses could be carried out, such as the impact of Japanese and English proficiency on the SCS test score, as well as the influence of the level of education on the three mental health markers.

**DATACAMP SUBMISSION**

In [23]:
-- Submission
SELECT
	stay,
	COUNT(*) AS count_int,
	ROUND(AVG(todep), 2) AS average_phq,
	ROUND(AVG(tosc), 2) AS average_scs,
	ROUND(AVG(toas), 2) AS average_asiss
FROM students 
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;

,stay,count_int,average_phq,average_scs,average_asiss
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80
